**Histórico dos dados dos fundos de investimentos**
* https://dados.cvm.gov.br/dataset/fi-doc-cda
* https://dados.cvm.gov.br/dataset/fi-doc-inf_diario

* Observação: muitas vezes, os arquivos excel do site acima não se atualizam rapidamente. Para ver a posição mais atualizada entrar no site:
* https://conteudo.cvm.gov.br/menu/regulados/fundos/consultas/fundos.html
---
O arquivo zipado 'cda_fi' contém vários arquivos excel. Os arquivos que eu tenho interesse são:

* cda_fi_BLC_1: arquivo que contém os 'Títulos Públicos';
* cda_fi_BLC_2: arquivo que contém as 'Cotas de Fundos';
* cda_fi_BLC_4: arquivo que contém as 'Ações', 'BDRs', 'Opções', 'Mercado futuro', 'Debêntures', 'Obrigações por ações' (posição vendida);
* cda_fi_BLC_7: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_BLC_8: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_PL: arquivo que contém o PL dos fundos de investimentos.

---
O arquivo zipado 'inf_diario_fi' contém apenas um arquivo excel.

In [2]:
import glob
import numpy as np
import os
import pandas as pd
from pandas import DataFrame
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import zipfile

## Extraindo os arquivos zipados

### Extração dos arquivos "cda_fi"

In [245]:
# Mês e ano escolhido
mes = '10'
ano = '2023'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}{mes}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}{mes}.csv', 
    f'cda_fi_BLC_2_{ano}{mes}.csv', 
    f'cda_fi_BLC_4_{ano}{mes}.csv', 
    f'cda_fi_BLC_7_{ano}{mes}.csv', 
    f'cda_fi_BLC_8_{ano}{mes}.csv',
    f'cda_fi_PL_{ano}{mes}.csv'
]

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

### Extração dos arquivos "inf_diario_fi"

In [235]:
# Descompactando apenas um arquivo

# Mês e ano escolhido
mes = '11'
ano = '2023'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{mes}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivo_a_extrair = [f'inf_diario_fi_{ano}{mes}.csv']

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivo_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

In [145]:
# Descompactando vários arquivos
for _ in ['01', '02', '03', '04']: 
    
    # Ano escolhido
    ano = '2024'

    # Nome do arquivo zipado
    zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{_}.zip'

    # Lista dos nomes dos arquivos que você deseja extrair
    arquivo_a_extrair = [f'inf_diario_fi_{ano}{_}.csv']

    # Caminho do diretório do diretório do arquivo não zipado
    destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

    # Abrindo o arquivo zipado
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extraindo os arquivos selecionados para um diretório específico
        for arquivo in arquivo_a_extrair:
            zip_ref.extract(arquivo, destino_dir)

            # Caminhos dos arquivos em formato csv e parquet
            csv_path = f'{destino_dir}//{arquivo}'
            parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

            # Lendo os arquivos csv
            df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

            # Transformando esses arquivos em parquet
            df.to_parquet(parquet_path)

    # Entrando para o diretório especificado
    os.chdir(destino_dir)

    # Encontrando todos os arquivos .csv no diretório
    arquivos_csv = glob.glob('*.csv')

    # Loop para deletar cada arquivo csv encontrado
    for arquivo in arquivos_csv:
        os.remove(arquivo)

## Funções

In [246]:
def rentabilidade_fundo(df: pd.DataFrame, cnpj: str):
    """
    Calcula a rentabilidade mensal e anual do fundo selecionado.

    Parameters:
    df: Dataframe que contem as cotas dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    df_ret_mensal: Dataframe dos retornos mensais.
    df_ret_anual: Dataframe dos retornos anuais.

    NOTE:Para calcular a rentabilidade mensal dos fundos - o site MaisRetorno utiliza última cota do mês anterior com a última cota do mês seguinte.
    Para calcular a rentabilidade anual dos fundos - o site MaisRetorno utiliza última cota do ano anterior com a última cota do ano seguinte.
    Por isso, eu tive que baixar a cota do mês 12/2022.
    """
    # Selecionando o fundo de investimentos específicos
    filt_cnpj = (df['CNPJ_FUNDO'] == cnpj)
    fundo_espec = df.loc[filt_cnpj]

    # Selecionando os últimos dias de cada mês
    last_days = fundo_espec.groupby(fundo_espec.index.to_period('M')).tail(1)
    # Selecionando apenas a coluna 'VL_QUOTA'
    last_days = last_days.loc[:, 'VL_QUOTA']
    # Calculando a rentabilidade do mês
    ret_mensal = round(last_days.pct_change()*100, 2)
    # Como eu estou utilizando a última cota do mês anterior com a a última cota do mês seguinte, a 1º rentabilidade ('2022-12-30') dessa série irá ser um NaN
    ret_mensal = ret_mensal.dropna()
    # Criando o df p/ os retornos mensais
    df_ret_mensal = pd.DataFrame(ret_mensal)
    # Renomeando a coluna 'VL_QUOTA'
    df_ret_mensal = df_ret_mensal.rename(columns={'VL_QUOTA':'ret_mensal'})

    # Transformando as poncentagens em taxa unitária
    df_ret_mensal['taxa_unit'] = 1 + (df_ret_mensal['ret_mensal'] / 100) 
    # Lista dos anos
    lst_years = df_ret_mensal.index.year.unique()
    # Transformando em string
    lst_years = lst_years.astype(str)
    # Calculando o retorno anual - aculumando as porcentagens de cada ano
    lst_ret_anual = []
    for _ in lst_years:
        ret_anual = round((df_ret_mensal.loc[_, 'taxa_unit'].agg(lambda x : x.prod()) -1) * 100, 2)
        lst_ret_anual.append(ret_anual)
    # Criando o df p/ os retornos anuais
    df_ret_anual = pd.DataFrame(lst_ret_anual, index=lst_years, columns=['ret_anual'])

    # Para não mostrar a coluna 'taxa_unit', selecionando apenas a coluna 'ret_mensal'
    df_ret_mensal_final = df_ret_mensal[['ret_mensal']]

    return df_ret_mensal_final, df_ret_anual


def open_cda_1(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_1'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_1'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'TP_TITPUB', 'DT_VENC']]

    # Mesclando as colunas 'TP_TITPUB' e 'DT_VENC' em apenas em uma coluna
    df['TP_TITPUB'] = df['TP_TITPUB'] + ' ' + df['DT_VENC']

    # Removendo a coluna 'DT_VENC'
    df = df.drop('DT_VENC', axis=1)

    # Renomeando a coluna 'TP_TITPUB' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_2/4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'TP_TITPUB':'CD_ATIVO'})

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_2(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_2'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_2'
    """
    # Lendo o arquivo. Adicionei o 'low_memory=False' para não dar o aviso -> DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'NM_FUNDO_COTA']]

    # Renomeando a coluna 'NM_FUNDO_COTA' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'NM_FUNDO_COTA':'CD_ATIVO'})

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_4(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_4'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_4'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ATIVO']]

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_7(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_7'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_7'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas.
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'EMISSOR']]

    # Renomeando a coluna 'EMISSOR' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs.
    df.rename(columns={"EMISSOR": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_8(path: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_8'.

    Parameters:
    path: caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_8'
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO']]

    # Renomeando a coluna 'DS_ATIVO' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs
    df.rename(columns={"DS_ATIVO": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    # Selecionando apenas o ativo 'BDR', porque neste arquivo também possui um ativo chamado 'Títulos Públicos', mas não é o principal 'Títulos Públicos', que está no 'cda_fi_BLC_1'
    filt_bdr = (df['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    df = df.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    
    return df


def pl_fundo(path: str, cnpj: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_PL'.
    
    Paramenters:
    path: caminho do arquivo.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Dataframe com o valor do patrimônio líquido do fundo de investimentos específico.
    """
    # Lendo o arquivo
    df = pd.read_parquet(path)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando o fundo de investimentos específicos
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # # Transformando os dtypes da coluna
    # fundo_espec['VL_PATRIM_LIQ'] = fundo_espec.loc[:, 'VL_PATRIM_LIQ'].astype(float)

    return fundo_espec['VL_PATRIM_LIQ']


def fundo_cnpj(df: pd.DataFrame, cnpj: str) -> DataFrame:
    """
    Separando o df do fundo de investimentos em várias categorias.

    Parameters:
    df: Dataframe que contém os ativos dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    """
    # Lendo o df concatenado
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # BDRs
    filt_bdr = (fundo_espec['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_bdr = fundo_espec.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_bdr = lambda x: (x / df_bdr['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_bdr['PORCENTAGEM'] = list(map(porcentagem_bdr, df_bdr['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_bdr =  df_bdr.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Investimentos Exterior
    filt_exterior = (fundo_espec['TP_APLIC'] == 'Investimento no Exterior')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_exterior = fundo_espec.loc[filt_exterior].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_exterior = lambda x: (x / df_exterior['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_exterior['PORCENTAGEM'] = list(map(porcentagem_exterior, df_exterior['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_exterior = df_exterior.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Cotas de Fundos
    filt_cotas_fundos = (fundo_espec['TP_APLIC'] == 'Cotas de Fundos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_cotas_fundos = fundo_espec.loc[filt_cotas_fundos].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada cota de fundo
    porcentagem_cotas = lambda x: (x / df_cotas_fundos['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_cotas_fundos['PORCENTAGEM'] = list(map(porcentagem_cotas, df_cotas_fundos['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_cotas_fundos = df_cotas_fundos.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Títulos públicos
    filt_titulos_pub = (fundo_espec['TP_APLIC'] == 'Títulos Públicos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_titulos_pub = fundo_espec.loc[filt_titulos_pub].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_titulos = lambda x: (x / df_titulos_pub['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_titulos_pub['PORCENTAGEM'] = list(map(porcentagem_titulos, df_titulos_pub['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_titulos_pub = df_titulos_pub.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Obrigações por ações e outros TVM recebidos em empréstimo
    filt_vendido_acoes = (fundo_espec['TP_APLIC'] == 'Obrigações por ações e outros TVM recebidos em empréstimo')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_vendido_acoes = fundo_espec.loc[filt_vendido_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_vendido = lambda x: (x / df_vendido_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_vendido_acoes['PORCENTAGEM'] = list(map(porcentagem_vendido, df_vendido_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_vendido_acoes = df_vendido_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return (
        df_acoes, 
        df_bdr, 
        df_exterior, 
        df_cotas_fundos, 
        df_titulos_pub, 
        df_vendido_acoes
    )


def fundo_cnpj_acoes(df: pd.DataFrame, cnpj: str) -> DataFrame:
    """
    Separando o df do fundo de investimento apenas na categoria de ações.

    Parameters:
    df: Dataframe que contém os ativos dos fundos.
    cnpj: cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    
    """
    # Lendo o df concatenado
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return df_acoes


def comparar_portfolios(df: pd.DataFrame, nome_fundo: str) -> str:
    """
    Comparação do portfólio - quais foram as ações que foram compradas e vendidas em relação ao mês anteirior.

    Pameters:
    df: DataFrame de cada mês do portfólio do fundo.
    nome_fundo: nome do fundo que está sendo analisado.

    Returns:
    Texto com as mudanças do portfólio.
    """
    for i in range(1, len(df)):
        mes_atual = df.iloc[i]
        mes_anterior = df.iloc[i - 1]
        
        data_atual = mes_atual['data']
        data_anterior = mes_anterior['data']
        
        acoes_atual = mes_atual['CD_ATIVO']
        acoes_anterior = mes_anterior['CD_ATIVO']
        
        vendidas = acoes_anterior - acoes_atual
        compradas = acoes_atual - acoes_anterior
        
        print(f'Comparando o portfólio de {data_anterior.strftime("%m/%Y")} e {data_atual.strftime("%m/%Y")}:')
        print(f'O {nome_fundo} vendeu as ações: {vendidas}')
        print(f'O {nome_fundo} comprou as ações: {compradas}')
        print('-' * 80)


def num_total_acoes(df: pd.DataFrame) -> pd.Series:
    """
    Parameters:
    df: DataFrame do portfólio do fundo.

    Returns:
    Número total de ações de cada mês do portfpolio.
    """
    num_total_acoes = df.groupby('data')['CD_ATIVO'].count()
    
    return num_total_acoes


def rank_top_5(df: pd.DataFrame) -> pd.Series:
    """
    Parameters:
    df: DataFrame do portfólio do fundo.
    
    Returns:
    rank_portfolio_fundo: rank das 5 maiores posições do fundo.
    """
    # Selecionando os valores únicos (datas) do df
    lst_data = df.index.unique()

    # Rank das 5 maiores posições do fundo
    lst_rank = []
    for _ in lst_data:
        rank = df.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
        lst_rank.append(rank)

    # Concatendo os dfs de rank
    rank_portfolio = pd.concat(lst_rank)
    rank_portfolio_fundo = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)

    return rank_portfolio_fundo


def plot_portfolio(df: pd.DataFrame, nome_fundo: str):
    """
    Parameters:
    df: Dataframe do portfólio do fundo selecionado.
    nome_fundo: nome do fundo selecionado.

    Returns:
    Plot do portfólio do fundo selecionado.
    """
    # Extraindo os meses únicos do índice
    months = df.index.to_period('M').unique()

    # Títulos dos subplots
    titulos = [f'Distribuição Percentual do Portfólio - {month.strftime("%m/%Y")}' for month in months]

    # Criando a figura com subplots
    fig = make_subplots(rows=len(months),
                        cols=1,
                        subplot_titles=titulos,
                        vertical_spacing=0.02  # Espaço entre os plots
    )

    # Iterando sobre os meses
    for idx, month in enumerate(months):
        # Extraindo os dados do portfólio para o mês atual
        portfolio_mes = df.loc[month.strftime('%Y-%m'), ['CD_ATIVO', 'PORCENTAGEM']]
        
        fig.add_trace(go.Bar(
            x=portfolio_mes['PORCENTAGEM'] * 100,
            y=portfolio_mes['CD_ATIVO'],
            orientation='h',
            name=month.strftime('%m/%Y')
        ), row=idx + 1, col=1)

    fig.update_layout(
        title=f'Portfólio do Fundo {nome_fundo}',
        height=6000,
        width=900
    )

    return fig.show()

In [247]:
# Formatando os arquivos parquet 'cda_fi'
df_cda_1 = open_cda_1(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_1_{ano}{mes}.parquet'
)
df_cda_2 = open_cda_2(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_2_{ano}{mes}.parquet'
)
df_cda_4 = open_cda_4(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_4_{ano}{mes}.parquet'
)
df_cda_7 = open_cda_7(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_7_{ano}{mes}.parquet'
)
df_cda_8 = open_cda_8(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_8_{ano}{mes}.parquet'
)

# Concatenando os dfs
df_ativos = pd.concat([df_cda_1, df_cda_2, df_cda_4, df_cda_7, df_cda_8])
df_ativos.tail()

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,VL_MERC_POS_FINAL,CD_ATIVO
165925,FI,44.434.985/0001-39,TRAFALGAR ORION FUNDO DE INVESTIMENTO - RENDA ...,2023-10-31,Brazilian Depository Receipt - BDR,BDR nível II,22.45,INBR32 -
90055,FI,20.895.433/0001-60,TERA FUNDO DE INVESTIMENTO MULTIMERCADO INVEST...,2023-10-31,Brazilian Depository Receipt - BDR,BDR nível I,8.58,MILK33 - LAEP - MILK33
87001,FI,18.990.723/0001-40,SICREDI FUNDO DE INVESTIMENTO ESPECIALMENTE CO...,2023-10-31,Brazilian Depository Receipt - BDR,BDR nível III,0.00,EXCO/EXCO32/BREXCOBDR000
159052,FI,42.656.788/0001-39,TBI FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO...,2023-10-31,Brazilian Depository Receipt - BDR,BDR nível I,0.00,XPBR DR1 - 09/10/2023
159053,FI,42.656.788/0001-39,TBI FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO...,2023-10-31,Brazilian Depository Receipt - BDR,BDR nível I,0.00,XPBR DR1 - 23/10/2023


In [197]:
# Formatando os arquivos parquet 'inf_diario_fi'

# Caminho da pasta dos arquivos 'inf_diario_fi'
path_inf_diario = 'C:\\Users\\vitor\\projetos_python\\python_b3\\historico-arquivos\\fundos_investimentos\\inf_diario_cvm\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos = glob.glob(path_inf_diario + '*.parquet')

# Lendo os arquivos parquet e juntando em apenas um df
lst_df = [pd.read_parquet(path) for path in paths_arquivos]

# Concatenando os dfs
df_inf_diario = pd.concat(lst_df, ignore_index=True)

# Selecionando apenas os 'Fundos de Investimentos'
filt_fi = df_inf_diario['TP_FUNDO'] == 'FI'
df_inf_diario = df_inf_diario.loc[filt_fi]

# Selecionando as principais colunas
df_inf_diario = df_inf_diario[['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC' , 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ']]

# Transformando os dtypes das colunas.
df_inf_diario['TP_FUNDO'] = df_inf_diario['TP_FUNDO'].astype(str)
df_inf_diario['CNPJ_FUNDO'] = df_inf_diario['CNPJ_FUNDO'].astype(str)
df_inf_diario['DT_COMPTC'] = pd.to_datetime(df_inf_diario['DT_COMPTC'], format='%Y-%m-%d')

# Transformando a coluna 'DT_COMPTC' em index
df_inf_diario = df_inf_diario.set_index('DT_COMPTC')
df_inf_diario.tail()

,TP_FUNDO,CNPJ_FUNDO,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ
DT_COMPTC,,,,,
2024-04-24,FI,97.929.213/0001-34,84757562.92,11.771143,84612563.16
2024-04-25,FI,97.929.213/0001-34,84725908.95,11.766658,84580324.04
2024-04-26,FI,97.929.213/0001-34,84999412.47,11.804625,84853242.58
2024-04-29,FI,97.929.213/0001-34,85055425.98,11.812336,84908669.75
2024-04-30,FI,97.929.213/0001-34,84742750.95,11.768756,84595408.13


In [198]:
# Quais são os tipos de ativos o fundo de investimento possui
lista_ativos = df_ativos['TP_APLIC'].drop_duplicates().to_list()
lista_ativos

['Operações Compromissadas',
 'Títulos Públicos',
 'Cotas de Fundos',
 'Ações',
 'Debêntures',
 'Certificado ou recibo de depósito de valores mobiliários',
 'Outros valores mobiliários registrados na CVM objeto de oferta pública',
 'Obrigações por ações e outros TVM recebidos em empréstimo',
 'Opções - Posições titulares',
 'Mercado Futuro - Posições compradas',
 'Mercado Futuro - Posições vendidas',
 'Ações e outros TVM cedidos em empréstimo',
 'Brazilian Depository Receipt - BDR',
 'Opções - Posições lançadas',
 'Compras a termo a receber',
 'Vendas a termo a receber',
 'Investimento no Exterior']

## Fundos de Investimentos

### Fundo Verde

* https://maisretorno.com/fundo/verde-am-acoes-master-fia-1

In [199]:
# Rentabilidade mensal
df_ret_mensal = rentabilidade_fundo(df=df_inf_diario, cnpj='16.929.553/0001-63')[0]
df_ret_mensal

,ret_mensal
DT_COMPTC,
2023-01-31,0.65
2023-02-28,-4.23
2023-03-31,-1.75
2023-04-28,-1.48
2023-05-31,5.05
2023-06-30,7.85
2023-07-31,2.09
2023-08-31,-2.25
2023-09-29,-0.18


In [248]:
# Rentabilidade anual
df_ret_anual = rentabilidade_fundo(df=df_inf_diario, cnpj='16.929.553/0001-63')[1]
df_ret_anual

,ret_anual
DT_COMPTC,
2023,11.57
2024,-10.44


In [249]:
verde_acoes, verde_bdr, verde_exterior, verde_cotas_fundos, verde_titulos_pub, verde_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='16.929.553/0001-63')
verde_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='16.929.553/0001-63'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(verde_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_bdr = round(pd.Series(verde_bdr['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_exterior = round(pd.Series(verde_exterior['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(verde_cotas_fundos['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_titulos_pub = round(pd.Series(verde_titulos_pub['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(verde_vendido_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
verde_acoes = pd.concat([verde_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
verde_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_acoes = verde_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
verde_bdr = pd.concat([verde_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
verde_bdr['data'] = data
# Definindo a coluna 'data' como o index
verde_bdr = verde_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
verde_exterior = pd.concat([verde_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
verde_exterior['data'] = data
# Definindo a coluna 'data' como o index
verde_exterior = verde_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
verde_cotas_fundos = pd.concat([verde_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
verde_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
verde_cotas_fundos = verde_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
verde_titulos_pub = pd.concat([verde_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
verde_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
verde_titulos_pub = verde_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
verde_vendido_acoes = pd.concat([verde_vendido_acoes, df_linha_pct_vendido_acoes ])
# Criando a coluna 'data' no df
verde_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_veverde_vendido_acoes =verde_vendido_acoes.set_index('data')

In [250]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//verde//fundo_verde_{ano}{mes}.xlsx') as writer:
    verde_acoes.to_excel(writer, sheet_name='acoes')
    verde_bdr.to_excel(writer, sheet_name='bdr')
    verde_exterior.to_excel(writer, sheet_name='exterior')
    verde_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    verde_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    verde_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [251]:
# Caminho da pasta do fundo Verde
path_verde = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\verde\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_verde = glob.glob(path_verde + '*.xlsx')

# Lendo cada arquivo excel
lst_dfs = []
for path in paths_arquivos_verde:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_verde = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_verde = portfolio_verde[portfolio_verde['VL_MERC_POS_FINAL'] != 0]
portfolio_verde = portfolio_verde.dropna()

# Definindo a coluna 'data' como o index
portfolio_verde = portfolio_verde.set_index('data')
portfolio_verde

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,SUZB3,1.296122e-01,45251996.00
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,RENT3,9.560451e-02,33378749.35
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,EQTL3,8.593888e-02,30004152.24
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,VBBR3,7.661107e-02,26747500.00
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,RAIL3,6.938032e-02,24223002.00
...,...,...,...,...
2023-10-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO EM...,LREN3,1.549761e-05,3616.70
2023-10-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO EM...,ABEV3,1.157211e-05,2700.60
2023-10-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO EM...,TCSA3,1.021376e-05,2383.60


In [252]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_verde.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Verde')

Comparando o portfólio de 01/2023 e 02/2023:
O Verde vendeu as ações: {'CMIG3', 'PETR4', 'RECV3', 'MODL3', 'YDUQ3', 'MULT3', 'CASH3', 'BEEF3', 'LJQQ3', 'ALSO3', 'SOMA3', 'TIMS3'}
O Verde comprou as ações: {'JBSS3', 'PSSA3', 'CPLE6', 'EZTC3', 'MRFG3', 'VALE3', 'LWSA3', 'EMBR3', 'EGIE3', 'CRFB3', 'BOAS3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Verde vendeu as ações: {'RDOR3', 'CIEL3', 'INTB3', 'EZTC3', 'GOAU4', 'MRFG3', 'SQIA3', 'RAIZ4', 'MTRE3', 'GGBR4', 'EMBR3', 'CSAN3', 'LREN3', 'GMAT3', 'BBAS3'}
O Verde comprou as ações: {'STBP3', 'ELET6', 'DIRR3', 'SMTO3', 'BRFS3', 'BEEF3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Verde vendeu as ações: {'USIM5', 'BRKM5', 'PSSA3', 'BRPR3', 'PETZ3', 'BRAP4', 'LWSA3', 'DIRR3', 'CBAV3', 'RRRP3', 'ELET3', 'ENJU3', 'CRFB3'}
O Verde comprou as ações: {'PETR3', 'PETR4', 'EZTC3', 'REC

In [253]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_verde)

data
2023-01-01    71
2023-02-01    70
2023-03-01    61
2023-04-01    60
2023-05-01    59
2023-06-01    58
2023-07-01    69
2023-08-01    51
2023-09-01    46
2023-10-01    43
Name: CD_ATIVO, dtype: int64

In [254]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_verde)

data
2023-01-01    [SUZB3, RENT3, EQTL3, VBBR3, RAIL3]
2023-02-01    [SUZB3, RENT3, RAIL3, VBBR3, EQTL3]
2023-03-01    [SUZB3, RAIL3, VBBR3, ASAI3, SBSP3]
2023-04-01    [SUZB3, RAIL3, VBBR3, RENT3, EQTL3]
2023-05-01    [SUZB3, RAIL3, EQTL3, VBBR3, SBSP3]
2023-06-01    [SUZB3, RENT3, RAIL3, EQTL3, VBBR3]
2023-07-01    [SUZB3, RAIL3, RENT3, VBBR3, ITUB4]
2023-08-01    [SUZB3, RAIL3, VBBR3, SBSP3, RENT3]
2023-09-01    [SUZB3, RAIL3, SBSP3, VBBR3, RENT3]
2023-10-01    [SUZB3, RAIL3, EQTL3, ITUB4, SBSP3]
Name: CD_ATIVO, dtype: object

In [255]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_verde, nome_fundo='Verde')

### Fundo Dynamo

* https://maisretorno.com/fundo/dynamo-cougar-master-fia

In [256]:
# Rentabilidade mensal
df_ret_mensal = rentabilidade_fundo(df=df_inf_diario, cnpj='37.916.879/0001-26')[0]
df_ret_mensal

,ret_mensal
DT_COMPTC,
2023-01-31,4.80
2023-02-28,-4.66
2023-03-31,-6.02
2023-04-28,-0.24
2023-05-31,12.20
2023-06-30,8.40
2023-07-31,5.00
2023-08-31,-6.35
2023-09-29,-0.77


In [257]:
# Rentabilidade anual
df_ret_anual = rentabilidade_fundo(df=df_inf_diario, cnpj='37.916.879/0001-26')[1]
df_ret_anual

,ret_anual
DT_COMPTC,
2023,24.92
2024,-2.81


In [258]:
dynamo_acoes, dynamo_bdr, dynamo_exterior, dynamo_cotas_fundos, dynamo_titulos_pub, dynamo_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='37.916.879/0001-26')
dynamo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet',
    cnpj='37.916.879/0001-26'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(dynamo_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_bdr = round(pd.Series(dynamo_bdr['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_exterior = round(pd.Series(dynamo_exterior['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(dynamo_cotas_fundos['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_titulos_pub = round(pd.Series(dynamo_titulos_pub['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(dynamo_vendido_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
dynamo_acoes = pd.concat([dynamo_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
dynamo_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_acoes = dynamo_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
dynamo_bdr = pd.concat([dynamo_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
dynamo_bdr['data'] = data
# Definindo a coluna 'data' como o index
dynamo_bdr = dynamo_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
dynamo_exterior = pd.concat([dynamo_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
dynamo_exterior['data'] = data
# Definindo a coluna 'data' como o index
dynamo_exterior = dynamo_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
dynamo_cotas_fundos = pd.concat([dynamo_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
dynamo_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
dynamo_codynamo_cotas_fundos =dynamo_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
dynamo_titulos_pub = pd.concat([dynamo_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
dynamo_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
dynamo_titulos_pub = dynamo_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
dynamo_vendido_acoes = pd.concat([dynamo_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
dynamo_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_vendido_acoes = dynamo_vendido_acoes.set_index('data')

In [259]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//dynamo//fundo_dynamo_{ano}{mes}.xlsx') as writer:
    dynamo_acoes.to_excel(writer, sheet_name='acoes')
    dynamo_bdr.to_excel(writer, sheet_name='bdr')
    dynamo_exterior.to_excel(writer, sheet_name='exterior')
    dynamo_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    dynamo_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    dynamo_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [260]:
# Caminho da pasta do fundo Dynamo
path_dynamo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\dynamo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_dynamo = glob.glob(path_dynamo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_dynamo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_dynamo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_dynamo = portfolio_dynamo[portfolio_dynamo['VL_MERC_POS_FINAL'] != 0]
portfolio_dynamo = portfolio_dynamo.dropna()

# Definindo a coluna 'data' como o index
portfolio_dynamo = portfolio_dynamo.set_index('data')
portfolio_dynamo

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,ENEV3,0.147659,9.885924e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,VBBR3,0.110783,7.417016e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,CSAN3,0.106247,7.113367e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,RENT3,0.100129,6.703738e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,NTCO3,0.093790,6.279359e+08
...,...,...,...,...
2023-10-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,OPCT3,0.001539,9.698096e+06
2023-10-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,SMFT3,0.001474,9.293339e+06
2023-10-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,GGBR4,0.001202,7.574240e+06


In [261]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_dynamo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Dynamo')

Comparando o portfólio de 01/2023 e 02/2023:
O Dynamo vendeu as ações: {'RENT3', 'PETR3', 'PETR4', 'ITUB4'}
O Dynamo comprou as ações: {'CCRO3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Dynamo vendeu as ações: {'GGBR4', 'ENJU3'}
O Dynamo comprou as ações: {'ASAI3', 'OPCT3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Dynamo vendeu as ações: set()
O Dynamo comprou as ações: {'GGBR4'}
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Dynamo vendeu as ações: set()
O Dynamo comprou as ações: {'RENT3', 'MOVI3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Dynamo vendeu as ações: {'B3SA3'}
O Dynamo comprou as ações: {'UGPA3', 'RAIZ4'}
-------------------------------

In [214]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_dynamo)

data
2023-01-01    30
2023-02-01    27
2023-03-01    27
2023-04-01    28
2023-05-01    30
2023-06-01    31
2023-07-01    31
2023-08-01    31
2023-09-01    31
2023-10-01    31
Name: CD_ATIVO, dtype: int64

In [262]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_dynamo)

data
2023-01-01    [ENEV3, VBBR3, CSAN3, RENT3, NTCO3]
2023-02-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-03-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-04-01    [ENEV3, CSAN3, VBBR3, RDOR3, NTCO3]
2023-05-01    [ENEV3, VBBR3, RDOR3, SUZB3, NTCO3]
2023-06-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-07-01    [ENEV3, RDOR3, VBBR3, NTCO3, CSAN3]
2023-08-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-09-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-10-01    [VBBR3, ENEV3, RDOR3, NTCO3, CSAN3]
Name: CD_ATIVO, dtype: object

In [263]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_dynamo, nome_fundo='Dynamo')

### Fundo IP

* https://maisretorno.com/fundo/ip-participacoes-master-fia-bdr-nivel-i

In [264]:
# Rentabilidade mensal
df_ret_mensal = rentabilidade_fundo(df=df_inf_diario, cnpj='11.435.298/0001-89')[0]
df_ret_mensal

,ret_mensal
DT_COMPTC,
2023-01-31,5.11
2023-02-28,-5.15
2023-03-31,0.56
2023-04-28,3.51
2023-05-31,3.03
2023-06-30,7.54
2023-07-31,4.21
2023-08-31,-0.94
2023-09-29,-1.87


In [265]:
# Rentabilidade anual
df_ret_anual = rentabilidade_fundo(df=df_inf_diario, cnpj='11.435.298/0001-89')[1]
df_ret_anual

,ret_anual
DT_COMPTC,
2023,30.94
2024,1.72


In [266]:
ip_acoes, ip_bdr, ip_exterior, ip_cotas_fundos, ip_titulos_pub, ip_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='11.435.298/0001-89')
ip_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='11.435.298/0001-89'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(ip_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_bdr = round(pd.Series(ip_bdr['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_exterior = round(pd.Series(ip_exterior['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(ip_cotas_fundos['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_titulos_pub = round(pd.Series(ip_titulos_pub['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(ip_vendido_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
ip_acoes = pd.concat([ip_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
ip_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_acoes = ip_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
ip_bdr = pd.concat([ip_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
ip_bdr['data'] = data
# Definindo a coluna 'data' como o index
ip_bdr = ip_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
ip_exterior = pd.concat([ip_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
ip_exterior['data'] = data
# Definindo a coluna 'data' como o index
ip_exterior = ip_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
ip_cotas_fundos = pd.concat([ip_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
ip_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
ip_cotas_fundos = ip_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
ip_titulos_pub = pd.concat([ip_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
ip_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
ip_titulos_pub = ip_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
ip_vendido_acoes = pd.concat([ip_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
ip_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_vendido_acoes = ip_vendido_acoes.set_index('data')

In [267]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//ip//fundo_ip_{ano}{mes}.xlsx') as writer:
    ip_acoes.to_excel(writer, sheet_name='acoes')
    ip_bdr.to_excel(writer, sheet_name='bdr')
    ip_exterior.to_excel(writer, sheet_name='exterior')
    ip_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    ip_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    ip_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [268]:
# Caminho da pasta do fundo IP
path_ip = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\ip\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_ip = glob.glob(path_ip + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_ip:
    df = pd.read_excel(path, sheet_name='bdr')
    lst_dfs.append(df)

# Concatendo os dfs de bdrs
portfolio_ip = pd.concat(lst_dfs, ignore_index=True)

# Retirando as bdrs que não tem nenhum dinheiro alocado e os NAN
portfolio_ip = portfolio_ip[portfolio_ip['VL_MERC_POS_FINAL'] != 0]
portfolio_ip = portfolio_ip.dropna()

# Definindo a coluna 'data' como o index
portfolio_ip = portfolio_ip.set_index('data')
portfolio_ip

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,GOGL34,0.186245,24025232.47
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,BERK34,0.155949,20117014.58
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,T1DG34,0.144290,18613037.52
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,LBRD34,0.133058,17164215.14
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,MSFT34,0.132903,17144173.56
...,...,...,...,...
2023-10-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,BERK34,0.217117,21768619.32
2023-10-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,LBRD34,0.157433,15784577.55
2023-10-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,MSCD34,0.151329,15172623.40


In [269]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_ip.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='IP')

Comparando o portfólio de 01/2023 e 02/2023:
O IP vendeu as ações: set()
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O IP vendeu as ações: set()
O IP comprou as ações: {'MSCD34'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O IP vendeu as ações: {'AMZO34', 'XPBR31'}
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O IP vendeu as ações: set()
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O IP vendeu as ações: set()
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 06/2023 e 07/2023:
O IP vendeu as ações

In [270]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_ip)

data
2023-01-01    8
2023-02-01    8
2023-03-01    9
2023-04-01    7
2023-05-01    7
2023-06-01    7
2023-07-01    6
2023-08-01    6
2023-09-01    6
2023-10-01    6
Name: CD_ATIVO, dtype: int64

In [271]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_ip)

data
2023-01-01    [GOGL34, BERK34, T1DG34, LBRD34, MSFT34]
2023-02-01    [BERK34, T1DG34, GOGL34, LBRD34, AMZO34]
2023-03-01    [GOGL34, BERK34, LBRD34, MSCD34, MSFT34]
2023-04-01    [GOGL34, BERK34, LBRD34, MSCD34, MSFT34]
2023-05-01    [GOGL34, BERK34, MSCD34, LBRD34, NFLX34]
2023-06-01    [GOGL34, BERK34, MSCD34, LBRD34, NFLX34]
2023-07-01    [GOGL34, BERK34, LBRD34, MSCD34, NFLX34]
2023-08-01    [GOGL34, BERK34, LBRD34, MSCD34, NFLX34]
2023-09-01    [GOGL34, BERK34, LBRD34, MSCD34, NFLX34]
2023-10-01    [GOGL34, BERK34, LBRD34, MSCD34, NFLX34]
Name: CD_ATIVO, dtype: object

In [272]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_ip, nome_fundo='IP')

### Fundo Squadra

* https://maisretorno.com/fundo/squadra-master-long-biased-fia
* Famosos pelos shorts de:
    * IRBR - 05/2018
    * AERI3 - 01/2021
    * Nubank

In [273]:
# Rentabilidade mensal
df_ret_mensal = rentabilidade_fundo(df=df_inf_diario, cnpj='09.412.648/0001-40')[0]
df_ret_mensal

,ret_mensal
DT_COMPTC,
2023-01-31,5.23
2023-02-28,-6.62
2023-03-31,-0.20
2023-04-28,3.92
2023-05-31,8.83
2023-06-30,8.13
2023-07-31,4.00
2023-08-31,2.24
2023-09-29,1.13


In [274]:
# Rentabilidade anual
df_ret_anual = rentabilidade_fundo(df=df_inf_diario, cnpj='09.412.648/0001-40')[1]
df_ret_anual

,ret_anual
DT_COMPTC,
2023,44.86
2024,-3.84


In [276]:
squadra_acoes, squadra_bdr, squadra_exterior, squadra_cotas_fundos, squadra_titulos_pub, squadra_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='09.412.648/0001-40')
squadra_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='09.412.648/0001-40'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(squadra_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_bdr = round(pd.Series(squadra_bdr['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_exterior = round(pd.Series(squadra_exterior['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(squadra_cotas_fundos['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_titulos_pub = round(pd.Series(squadra_titulos_pub['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(squadra_vendido_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
squadra_acoes = pd.concat([squadra_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_acoes = squadra_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
squadra_bdr = pd.concat([squadra_bdr, df_linha_pct_bdr], ignore_index=True)
# Criando a coluna 'data' no df
squadra_bdr['data'] = data
# Definindo a coluna 'data' como o index
squadra_bdr = squadra_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
squadra_exterior = pd.concat([squadra_exterior, df_linha_pct_exterior], ignore_index=True)
# Criando a coluna 'data' no df
squadra_exterior['data'] = data
# Definindo a coluna 'data' como o index
squadra_exterior = squadra_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
squadra_cotas_fundos = pd.concat([squadra_cotas_fundos, df_linha_pct_cotas_fundos], ignore_index=True)
# Criando a coluna 'data' no df
squadra_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
squadra_cotas_fundos = squadra_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
squadra_titulos_pub = pd.concat([squadra_titulos_pub, df_linha_pct_titulos_pub], ignore_index=True)
# Criando a coluna 'data' no df
squadra_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
squadra_titulos_pub = squadra_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
squadra_vendido_acoes = pd.concat([squadra_vendido_acoes, df_linha_pct_vendido_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_vendido_acoes = squadra_vendido_acoes.set_index('data')

In [277]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//squadra//fundo_squadra_{ano}{mes}.xlsx') as writer:
    squadra_acoes.to_excel(writer, sheet_name='acoes')
    squadra_bdr.to_excel(writer, sheet_name='bdr')
    squadra_exterior.to_excel(writer, sheet_name='exterior')
    squadra_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    squadra_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    squadra_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [278]:
# Caminho da pasta do fundo Squadra
path_squadra = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\squadra\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_squadra = glob.glob(path_squadra + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_squadra:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_squadra = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_squadra = portfolio_squadra[portfolio_squadra['VL_MERC_POS_FINAL'] != 0]
portfolio_squadra = portfolio_squadra.dropna()

# Definindo a coluna 'data' como o index
portfolio_squadra = portfolio_squadra.set_index('data')
portfolio_squadra

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,EQTL3,0.247564,4.684867e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,PRIO3,0.101156,1.914264e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,RAIL3,0.098252,1.859313e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,GMAT3,0.086300,1.633124e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,GGPS3,0.078096,1.477881e+08
...,...,...,...,...
2023-10-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,RDOR3,0.003681,6.973250e+06
2023-10-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,MGLU3,0.002335,4.422911e+06
2023-10-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,PETZ3,0.001786,3.384275e+06


In [279]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_squadra.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Squadra')

Comparando o portfólio de 01/2023 e 02/2023:
O Squadra vendeu as ações: set()
O Squadra comprou as ações: {'MGLU3', 'PETZ3', 'VIIA3', 'INTB3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Squadra vendeu as ações: {'B3SA3', 'ENEV3', 'PETZ3'}
O Squadra comprou as ações: {'WEGE3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Squadra vendeu as ações: {'LWSA3'}
O Squadra comprou as ações: {'PETZ3', 'CXSE3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Squadra vendeu as ações: {'CXSE3', 'CRFB3', 'MULT3', 'ITUB4'}
O Squadra comprou as ações: {'RDOR3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Squadra vendeu as ações: {'CRDE3'}
O Squadra comprou as ações: {'ASAI3',

In [280]:
# Número total de ações de cada mês do portfólio
num_total_acoes(df=portfolio_squadra)

data
2023-01-01    21
2023-02-01    25
2023-03-01    23
2023-04-01    24
2023-05-01    21
2023-06-01    23
2023-07-01    23
2023-08-01    27
2023-09-01    26
2023-10-01    26
Name: CD_ATIVO, dtype: int64

In [281]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_squadra)

data
2023-01-01    [EQTL3, PRIO3, RAIL3, GMAT3, GGPS3]
2023-02-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-03-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-04-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-05-01    [EQTL3, RAIL3, GMAT3, PRIO3, GGPS3]
2023-06-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-07-01    [EQTL3, PRIO3, RAIL3, GGPS3, GMAT3]
2023-08-01    [RAIL3, EQTL3, GGPS3, GMAT3, PRIO3]
2023-09-01    [RAIL3, EQTL3, GGPS3, GMAT3, UGPA3]
2023-10-01    [RAIL3, EQTL3, GGPS3, VBBR3, GMAT3]
Name: CD_ATIVO, dtype: object

In [282]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_squadra, nome_fundo='Squadra')

### Fundo Guepardo

* https://maisretorno.com/fundo/guepardo-institucional-master-fia

In [283]:
# Rentabilidade mensal
df_ret_mensal = rentabilidade_fundo(df=df_inf_diario, cnpj='14.213.077/0001-54')[0]
df_ret_mensal

,ret_mensal
DT_COMPTC,
2023-01-31,-0.74
2023-02-28,-3.55
2023-03-31,-0.86
2023-04-28,6.05
2023-05-31,14.29
2023-06-30,9.84
2023-07-31,5.92
2023-08-31,-2.77
2023-09-29,1.84


In [284]:
# Rentabilidade anual
df_ret_anual = rentabilidade_fundo(df=df_inf_diario, cnpj='14.213.077/0001-54')[1]
df_ret_anual

,ret_anual
DT_COMPTC,
2023,49.26
2024,-3.91


In [285]:
guepardo_acoes = fundo_cnpj_acoes(df=df_ativos, cnpj='14.213.077/0001-54')
guepardo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='14.213.077/0001-54'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(guepardo_acoes['VL_MERC_POS_FINAL'].sum() / guepardo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
guepardo_acoes = pd.concat([guepardo_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
guepardo_acoes['data'] = data
# Definindo a coluna 'data' como o index
guepardo_acoes = guepardo_acoes.set_index('data')

In [286]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//guepardo//fundo_guepardo_{ano}{mes}.xlsx') as writer:
    guepardo_acoes.to_excel(writer, sheet_name='acoes')
    

In [287]:
# Caminho da pasta do fundo Guepardo
path_guepardo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\arquivos_fundos\\guepardo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_guepardo = glob.glob(path_guepardo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_guepardo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado  e os NAN
portfolio_guepardo = portfolio_guepardo[portfolio_guepardo['VL_MERC_POS_FINAL'] != 0]
portfolio_guepardo = portfolio_guepardo.dropna()

# Definindo a coluna 'data' como o index
portfolio_guepardo = portfolio_guepardo.set_index('data')
portfolio_guepardo

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,VULC3,0.310343,2.492254e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ITUB4,0.172180,1.382715e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,UGPA3,0.159579,1.281520e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,RAIL3,0.150022,1.204777e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ANIM3,0.111362,8.943062e+07
...,...,...,...,...
2023-10-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,FLRY3,0.015946,2.189535e+07
2023-10-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ITUB3,0.012293,1.687917e+07
2023-10-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,VAMO3,0.009610,1.319546e+07


In [288]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_guepardo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Guepardo')

Comparando o portfólio de 01/2023 e 02/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Guepardo vendeu as ações: {'DXCO3'}
O Guepardo comprou as ações: {'GGBR4'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Guepardo vendeu as ações: {'MLAS3', 'GMAT3'}
O Guepardo comprou as ações: {'FLRY3'}
--------------------------------------------------------------------------------


In [289]:
# Número total de ações de cada mês do portfpolio
num_total_acoes(df=portfolio_guepardo)

data
2023-01-01    11
2023-02-01    11
2023-03-01    11
2023-04-01    11
2023-05-01    11
2023-06-01    10
2023-07-01    11
2023-08-01    11
2023-09-01    10
2023-10-01    12
Name: CD_ATIVO, dtype: int64

In [290]:
# Rank das 5 maiores ações do fundo
rank_top_5(df=portfolio_guepardo)

data
2023-01-01    [VULC3, ITUB4, UGPA3, RAIL3, ANIM3]
2023-02-01    [VULC3, ITUB4, UGPA3, RAIL3, ANIM3]
2023-03-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-04-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-05-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-06-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-07-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-08-01    [VULC3, UGPA3, ITUB4, RAIL3, GMAT3]
2023-09-01    [VULC3, UGPA3, ITUB4, RAIL3, GMAT3]
2023-10-01    [VULC3, UGPA3, ITUB4, RAIL3, GMAT3]
Name: CD_ATIVO, dtype: object

In [291]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_guepardo, nome_fundo='Guepardo')